In [7]:
import numpy as np
import pandas as pd
import graphlab 
import sklearn 

In [9]:
sales=graphlab.SFrame('kc_house_data.gl/')
sales_pd=pd.read_csv('kc_house_data.csv')

In [10]:
train_data_gl,test_data_gl=sales.random_split(.8,seed=0)
## a function to get split test and train datasets for pandas dataframe
## its input is a pandas dataframe and out put are two dataframes
def get_random_split_pd(data,seed=0.8):
    random=np.random.rand(len(data))<seed
    return data[random],data[~random]
train_data_sk,test_data_sk=get_random_split_pd(sales_pd)

In [13]:
test_features=['sqft_living','bedrooms','bathrooms']

#from sklearn.linear_model import LinearRegression

#linreg=LinearRegression()
example_model_gl=graphlab.linear_regression.create(train_data_gl,target='price',features=test_features,validation_set=None)
#example_model_pd=linreg.fit()

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.033522     | 4146407.600631     | 258679.804477 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [14]:
print example_model_gl.get("coefficients")

+-------------+-------+----------------+---------------+
|     name    | index |     value      |     stderr    |
+-------------+-------+----------------+---------------+
| (intercept) |  None | 87910.0724924  |  7873.3381434 |
| sqft_living |  None | 315.403440552  | 3.45570032585 |
|   bedrooms  |  None | -65080.2155528 | 2717.45685442 |
|  bathrooms  |  None | 6944.02019265  | 3923.11493144 |
+-------------+-------+----------------+---------------+
[4 rows x 4 columns]



In [16]:
example_predictions_gl=example_model_gl.predict(test_data_gl)
print example_predictions_gl[0],test_data_gl[0]["price"]

350640.366016 310000.0


In [19]:
def get_residual_sum_of_squares(model,data,outcome):
    estimates=model.predict(data)
    return sum((np.array(estimates)-np.array(outcome))**2)

print get_residual_sum_of_squares(example_model_gl,test_data_gl,test_data_gl["price"])

273761538330190.66
